## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt


In [3]:
# Import API key
from config import g_key


In [4]:
# Configure gmaps API key

gmaps.configure(api_key = g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kahului,US,20.8947,-156.4700,76.84,77,20,3.44,few clouds
1,1,Port Elizabeth,ZA,-33.9180,25.5701,67.10,78,41,4.23,scattered clouds
2,2,Atuona,PF,-9.8000,-139.0333,78.01,76,33,21.88,scattered clouds
3,3,Ginir,ET,7.1333,40.7000,62.20,42,67,3.58,broken clouds
4,4,Kavieng,PG,-2.5744,150.7967,80.44,77,99,8.39,overcast clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

# First let's create a prompt that asks the  customer for the minimum temperature preference.
min_temp = float(input("What is the minimum temperature you would prefer for your trip? "))
# Second let's creat a prompt that asks the customer for the maximum temperature preference. 
max_temp = float(input("What is the maximum temperature you would prefer for your trip? "))


What is the minimum temperature you would prefer for your trip? 30
What is the maximum temperature you would prefer for your trip? 85


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                       
preferred_cities_df.head()


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kahului,US,20.8947,-156.4700,76.84,77,20,3.44,few clouds
1,1,Port Elizabeth,ZA,-33.9180,25.5701,67.10,78,41,4.23,scattered clouds
2,2,Atuona,PF,-9.8000,-139.0333,78.01,76,33,21.88,scattered clouds
3,3,Ginir,ET,7.1333,40.7000,62.20,42,67,3.58,broken clouds
4,4,Kavieng,PG,-2.5744,150.7967,80.44,77,99,8.39,overcast clouds


In [8]:
preferred_cities_df.notnull().sum()


City_ID                558
City                   558
Country                554
Lat                    558
Lng                    558
Max Temp               558
Humidity               558
Cloudiness             558
Wind Speed             558
Current Description    558
dtype: int64

In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()


City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_hotel_df = preferred_cities_df.dropna()


In [11]:
clean_hotel_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kahului,US,20.8947,-156.4700,76.84,77,20,3.44,few clouds
1,1,Port Elizabeth,ZA,-33.9180,25.5701,67.10,78,41,4.23,scattered clouds
2,2,Atuona,PF,-9.8000,-139.0333,78.01,76,33,21.88,scattered clouds
3,3,Ginir,ET,7.1333,40.7000,62.20,42,67,3.58,broken clouds
4,4,Kavieng,PG,-2.5744,150.7967,80.44,77,99,8.39,overcast clouds


In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_hotel_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kahului,US,76.84,few clouds,20.8947,-156.4700,
1,Port Elizabeth,ZA,67.10,scattered clouds,-33.9180,25.5701,
2,Atuona,PF,78.01,scattered clouds,-9.8000,-139.0333,
3,Ginir,ET,62.20,broken clouds,7.1333,40.7000,
4,Kavieng,PG,80.44,overcast clouds,-2.5744,150.7967,
6,Bredasdorp,ZA,64.27,overcast clouds,-34.5322,20.0403,
7,Vaini,TO,69.96,scattered clouds,-21.2000,-175.2000,
9,Klaksvik,FO,43.72,broken clouds,62.2266,-6.5890,
10,Arraial Do Cabo,BR,69.30,clear sky,-22.9661,-42.0278,
11,Yellowknife,CA,37.42,clear sky,62.4560,-114.3525,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # 6e. Make request and retrieve the JSON data from the search. 
    
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found... skipping.")
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [14]:
# 7. Drop the rows where there is no Hotel

hotel_df['Hotel Name'].replace('', np.nan, inplace=True)


In [15]:
hotel_df.dropna(subset=['Hotel Name'], inplace=True)


In [16]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kahului,US,76.84,few clouds,20.8947,-156.4700,Maui Seaside Hotel
1,Port Elizabeth,ZA,67.10,scattered clouds,-33.9180,25.5701,39 On Nile Guest House
2,Atuona,PF,78.01,scattered clouds,-9.8000,-139.0333,Villa Enata
3,Ginir,ET,62.20,broken clouds,7.1333,40.7000,Sura Dare Village
4,Kavieng,PG,80.44,overcast clouds,-2.5744,150.7967,Nusa Island Retreat


In [17]:
# 8a. Create the output File (CSV)

# Create the output file (CSV).
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""



In [19]:
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]



In [20]:
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)



# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))